<a href="https://colab.research.google.com/github/karnamohit/kranka_ucm/blob/master/tf1_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all the useful libraries...

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Checking the version of TensorFlow, NumPy...

In [0]:
print('**********************************************')
print('TensorFlow version info:\t',tf.__version__)
#!pip show tensorflow
print('NumPy version info:\t \t',np.__version__)
print('**********************************************')

Need to import data...

In [0]:
!pwd

Make up data (2x2, sequentially indexed matrices)...

In [0]:
series_data = np.random.rand(100,2,2)      # "time"-series data for a 2x2 evolving matrix with 100 time-steps, randomly initialized
timesteps = int(series_data.shape[0])
mat_row = int(series_data.shape[1])
mat_col = int(series_data.shape[2])

Define a sequence (multiple consecutive 2x2 matrices) size, ```n_seq```, to be fed in as input (equivalent to the amount of memory in previous time-steps, in our case), and the amount of overlap, ```n_overlap``` between any two adjacent input sequences...

In [0]:
n_seq = 3
n_overlap = 2

if (n_overlap >= n_seq):
  raise Exception("n_overlap must be less than n_seq")

Build tensor of sequences, call it ```seq_series_data```...

In [0]:
seq_series_data = np.zeros(((timesteps - n_seq), mat_row, mat_col), np.float64)
for i in (timesteps - n_seq):
  

Use ```tf.data``` for storing input data (```series_data``` in the example above)

In [0]:
input_slices = tf.data.Dataset.from_tensor_slices(series_data)
next_slice_item = input_slices.make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


Run a ```tf.Session()``` (within the context manager) to verify sequential printing of the evolving 2x2 matrix...

In [0]:
with tf.Session() as sess:
  while True:
    try:
      print(sess.run(next_slice_item))
    except tf.errors.OutOfRangeError:
      break

Define batches of the ```input_slices``` (saves memory when building the computational graph)

In [0]:
input_batches = input_slices.batches(N,drop_remainder=True)

Set up placeholders for input, true output, predicted output...

In [0]:
x = tf.placeholder(tf.float)